In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

In [4]:
import sys
sys.path.append('/home/zhuf/CodingSpace/JupyterNotebook/machine_learning')
from playML.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

# 线性回归的解析解

要求解的参数$\theta$，第一个元素是截距，后面是X各个特征对应的系数

$X_b$是在X上扩展一列全为1元素后得到的矩阵

$$\theta = (X_b^TX_b)^{−1}X_b^Ty$$

预测值：

$$\hat y = X_b\theta$$

- 上述解称为正规方程解(Normal Equation)
- 线性回归算法能够得到最终的解析解，这在机器学习算法中是比较少见的，一般机器学习算法很难得到最终解析解
- 可以直接使用numpy提供的矩阵运算获得最终结果，但是实际上这是比较低效的计算方法，尤其是当计算规模较大的时候（无论是行数多还是列数多）
- 缺点：正规方程解的计算复杂度达到了$O(n^3)$,即使进行计算优化，也有$O(n^{2.4})$的复杂度
- 优点：是不需要对数据做归一化处理
- Sklearn其实是使用梯度下降算法进行求解的

# 使用自己的线性回归模型

In [6]:
from playML.LinearRegression import  LinearRegression

In [7]:
reg = LinearRegression()

In [8]:
reg.fit_normal(X_train, y_train)

LinearRegression_MyOwnVersion

In [9]:
reg.score(X_test, y_test)

0.8129794056212918

In [10]:
reg.intercept_

34.1173997232074

In [11]:
reg.coef_

array([-1.20354261e-01,  3.64423279e-02, -3.61493155e-02,  5.12978140e-02,
       -1.15775825e+01,  3.42740062e+00, -2.32311760e-02, -1.19487594e+00,
        2.60101728e-01, -1.40219119e-02, -8.35430488e-01,  7.80472852e-03,
       -3.80923751e-01])

# Sklearn中的线性回归

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
lin_reg = LinearRegression()

In [14]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
lin_reg.score(X_test, y_test)

0.812979405621281

In [16]:
lin_reg.intercept_

34.11739972322955

In [17]:
lin_reg.coef_

array([-1.20354261e-01,  3.64423279e-02, -3.61493155e-02,  5.12978140e-02,
       -1.15775825e+01,  3.42740062e+00, -2.32311760e-02, -1.19487594e+00,
        2.60101728e-01, -1.40219119e-02, -8.35430488e-01,  7.80472852e-03,
       -3.80923751e-01])

# 使用KNN回归

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [19]:
sd_scaler = StandardScaler()

In [20]:
sd_scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
X_train_scale = sd_scaler.transform(X_train)
X_test_scale = sd_scaler.transform(X_test)

In [22]:
knn_reg = KNeighborsRegressor()

In [23]:
knn_reg.fit(X_train_scale,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [24]:
knn_reg.score(X_test_scale,y_test)

0.847923904906593

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "weights": ["uniform"],
        "n_neighbors": [i for i in range(1, 11)]
    },
    {
        "weights": ["distance"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1,6)]
    }
]

knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train_scale, y_train)

/home/zhuf/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.8s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [27]:
grid_search.best_params_

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [30]:
# 注意这个得分是交叉验证计算的得分，而不是使用R方获得的得分
grid_search.best_score_

0.7991317809214437

In [29]:
grid_search.best_estimator_.score(X_test_scale, y_test)

0.8814052328522632

注意在评价模型性能优劣的时候，要注意评价指标的一致性
在我们上面的例子中，KNN回归使用了交叉验证的评判标准获取到了最优参数，而不是使用测试数据的R方